# Modèle CNN prédiction de points à la fin de la saison

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [53]:
#print le dossier où se trouve le fichier
import os
print(os.getcwd())

os.chdir("D:\\Moi\\Travail\\ESME\\INGE3\\SEMESTRE 9\\UE5 Projet\\RSPL\\src\\")
print(os.getcwd())

data = pd.read_csv("../data/cleaned_merged_seasons.csv")



D:\Moi\Travail\ESME\INGE3\SEMESTRE 9\UE5 Projet\RSPL\src
D:\Moi\Travail\ESME\INGE3\SEMESTRE 9\UE5 Projet\RSPL\src


C:\Users\bourd\AppData\Local\Temp\ipykernel_8844\1080581989.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/cleaned_merged_seasons.csv")


In [54]:
print(data.head())


  season_x                name position team_x  assists  bonus  bps  \
0  2016-17     Aaron Cresswell      DEF    NaN        0      0    0   
1  2016-17        Aaron Lennon      MID    NaN        0      0    6   
2  2016-17        Aaron Ramsey      MID    NaN        0      0    5   
3  2016-17  Abdoulaye Doucouré      MID    NaN        0      0    0   
4  2016-17        Adam Forshaw      MID    NaN        0      0    3   

   clean_sheets  creativity  element  ...  team_h_score  threat  total_points  \
0             0         0.0      454  ...           2.0     0.0             0   
1             0         0.3      142  ...           1.0     0.0             1   
2             0         4.9       16  ...           3.0    23.0             2   
3             0         0.0      482  ...           1.0     0.0             0   
4             0         1.3      286  ...           1.0     0.0             1   

   transfers_balance  transfers_in transfers_out  value  was_home  \
0                

In [55]:
# valeurs manquantes
data = data.dropna(subset=['team_x'])

# Normalise
features_to_normalize = ["creativity", "ict_index", "influence", "threat"]
data[features_to_normalize] = data[features_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Encode
data_encoded = pd.get_dummies(data, columns=["season_x", "position", "team_x", "kickoff_time", "opp_team_name"], drop_first=True)

# data type int
data_encoded['was_home'] = data_encoded['was_home'].astype(int)

# Retirer la colonne "name"
player_names = data_encoded['name'].reset_index(drop=True)
data_encoded = data_encoded.drop('name', axis=1)

In [56]:
print(player_names)

0                  Aaron Connolly
1                 Aaron Cresswell
2                      Aaron Mooy
3                  Aaron Ramsdale
4              Abdoulaye Doucouré
                   ...           
76312                Oliver Skipp
76313              Ryan Sessegnon
76314                Ashley Young
76315    Jeremy Sarmiento Morante
76316              Philip Billing
Name: name, Length: 76317, dtype: object


In [57]:
# features (X) et cible (y)
X = data_encoded.copy()

# Ajouter les colonnes 'team_x' et 'position' à X
columns_to_add = ['team_x', 'position']
if all(col in data_encoded.columns for col in columns_to_add):
    X[columns_to_add] = data_encoded[columns_to_add]

# Supprimer la colonne 'total_points' de X
X = X.drop('total_points', axis=1)

# Cible (y)
y = data_encoded['total_points']

In [58]:
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(X, y, player_names, test_size=0.2, random_state=42)

In [59]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models

In [60]:
# Redimensionner les données pour les modèles CNN (ajuster la forme en fonction de vos besoins)
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

In [61]:
# Créer le modèle CNN
model = models.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

In [62]:

# Compiler le modèle
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Faire des prédictions
y_pred = model.predict(X_test)


Epoch 1/10
1527/1527 [==============================] - 48s 31ms/step - loss: 1626388.2500 - mae: 157.3848 - val_loss: 24069.5508 - val_mae: 29.9702
Epoch 2/10
1527/1527 [==============================] - 47s 31ms/step - loss: 55763.8086 - mae: 32.1979 - val_loss: 1986727.7500 - val_mae: 472.0025
Epoch 3/10
1527/1527 [==============================] - 45s 30ms/step - loss: 5643.5312 - mae: 2.1777 - val_loss: 7.3918 - val_mae: 1.2608
Epoch 4/10
1527/1527 [==============================] - 45s 30ms/step - loss: 7.3441 - mae: 1.2594 - val_loss: 7.3082 - val_mae: 1.2677
Epoch 5/10
1527/1527 [==============================] - 44s 29ms/step - loss: 7.2238 - mae: 1.2693 - val_loss: 7.1425 - val_mae: 1.2819
Epoch 6/10
1527/1527 [==============================] - 44s 29ms/step - loss: 7.0000 - mae: 1.2899 - val_loss: 6.8518 - val_mae: 1.3082
Epoch 7/10
1527/1527 [==============================] - 43s 28ms/step - loss: 6.6458 - mae: 1.3151 - val_loss: 6.4107 - val_mae: 1.3259
Epoch 8/10
1527/152

In [63]:

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R2 Score: {r2}')

Mean Squared Error: 5.67501203115329
Mean Absolute Error: 1.5574145391817737
R2 Score: 0.03866827630975134
